In [6]:
from docplex.mp.model import Model
import numpy as np
from RandomNumberGenerator import RandomNumberGenerator
import math
import os

In [7]:
class RandomNumberGenerator:
    def __init__(self, seedVaule = None):
        self.__seed = seedVaule

    def nextInt(self, low, high):
        m = 2147483647
        a = 16807
        b = 127773
        c = 2836
        
        k = int(self.__seed / b)
        self.__seed = a * (self.__seed % b) - k * c

        if self.__seed < 0:
            self.__seed = self.__seed + m

        value_0_1 = self.__seed
        value_0_1 =  value_0_1/m
        
        return low + int(math.floor(value_0_1 * (high - low + 1)))
    

    def nextFloat(self, low, high):
        low *= 100000
        high *= 100000
        val = self.nextInt(low, high) / 100000.0
        
        return val
    

# INPUT VARIABLES: ---------------
seed = 42
r = RandomNumberGenerator(seed)
min_n = 2
max_n = 50
# ----------------------------------

def get_var(n: int):
    c = [0] * n
    w = [0] * n
    v = [0] * n
    for i in range(n):
        c[i] = r.nextFloat(1, 10)
        w[i] = r.nextFloat(1, 10)
        v[i] = r.nextFloat(1, 10)

    B = r.nextFloat(n, 4*n) 
    return c, w, v, B


In [8]:
import re
import io

def get_ticks(s: str):
    print(s)
    match = re.search(r'\(([-+]?\d*\.?\d+)\s+ticks\)', s)
    if match:
        return float(match.group(1))
    else:
        return None
    
    

In [9]:

def solve_5_3(n):
    model = Model(name='example')
    # init values
    c, w, v, B = get_var(n)


    x = [model.continuous_var(name=f'x_{i}') for i in range(n)]
    y = [model.continuous_var(name=f'y_{i}') for i in range(n)]

    # constraints
    for i in range(n):
        model.add_constraint(x[i] + y[i] <= 1)
        model.add_constraint(x[i] >= 0)
        model.add_constraint(y[i] >= 0)
        
    
    model.add_constraint(model.sum(x[i] * w[i] for i in range(n)) <= B)
    model.add_constraint(model.sum(y[i] * v[i] for i in range(n)) <= B)
    
    # maximize
    total_cost = model.sum((x[i] + y[i]) * c[i] for i in range(n))
    model.maximize(total_cost)    
    
    log_buffer = io.StringIO()
    solution = model.solve(log_output=log_buffer)
    
    
    if not solution:
        raise Exception("No solution found")
    
    solution_x = [x[i].solution_value for i in range(n)]
    solution_y = [y[i].solution_value for i in range(n)]
    
    total_cost = solution.get_objective_value()
    
    # for i in range(n):
    #     print(f"[{i}] x_{i}: {solution_x[i]}, y_{i}: {solution_y[i]}")
        
    return total_cost, get_ticks(log_buffer.getvalue())

In [10]:
import pandas as pd


reps = 20
ns = range(5, 331, 5)

result = []
for n in ns:
    res = []
    print(f"n = {n}")
    for i in range(reps):
        solution, ticks = solve_5_3(n)
        res.append(ticks)
    result.append([n, np.mean(res), np.std(res)])
    
df = pd.DataFrame(result, columns=['n', 'ticks_mean', 'ticks_std'])
df.to_csv('5_3.csv', index=False)
print(df)

n = 5
Version identifier: 22.1.2.0 | 2024-12-09 | 8bd2200c8
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 10 rows and 0 columns.
Reduced LP has 7 rows, 10 columns, and 20 nonzeros.
Presolve time = 0.00 sec. (0.01 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =            26.574376
Iteration:     3   Dual objective     =            39.448513

Version identifier: 22.1.2.0 | 2024-12-09 | 8bd2200c8
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 10 rows and 0 columns.
Reduced LP has 7 rows, 10 columns, and 20 nonzeros.
Presolve time = 0.00 sec. (0.01 ticks)

Iteration log . . .
Iteration:     1   Dual infeasibility =            26.020826
Iteration:     3   Dual objective     =            21.357685

Version identifier: 22.1.2.0 | 2024-12-09 | 8bd2200c8
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 10 rows and 0 colum